**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data files**

In [2]:
FOLDER = "annotation_ccre"

In [3]:
FD_REG = file.path(
    FD_RES, 
    "results", 
    "region", 
    "annotation_enhancer_merge")

FD_ANT = file.path(
    FD_RES, 
    "results", 
    "region", 
    FOLDER)

In [4]:
fdiry  = FD_REG
fnames = dir(fdiry)
for(fname in fnames){print(fname)}

[1] "description.tsv"
[1] "enhancer_merge.concat.starrmpra_crispr_atac.active.bed.gz"
[1] "enhancer_merge.concat.starrmpra_crispr_atac.active.merge.bed.gz"
[1] "enhancer_merge.concat.starrmpra_crispr_atac.total.bed.gz"
[1] "enhancer_merge.concat.starrmpra_crispr_atac.total.merge.bed.gz"
[1] "enhancer_table.starrmpra_crispr_atac.active.merge.tsv"
[1] "enhancer_table.starrmpra_crispr_atac.total.merge.tsv"
[1] "region_active"
[1] "region_total"
[1] "summary"
[1] "tmp"


In [5]:
fdiry  = FD_ANT
fnames = dir(fdiry)
for(fname in fnames){print(fname)}

[1] "ccre_v3.bed.gz"
[1] "ccre_v4.bed.gz"
[1] "description.tsv"
[1] "summary"


In [6]:
fdiry = file.path(FD_REG, "region_active", FOLDER)
fnames = dir(fdiry)
for(fname in fnames){print(fname)}

[1] "peak.annotation.ccre_v3.bed.gz"
[1] "peak.annotation.ccre_v4.bed.gz"


## Import data

In [7]:
### set file paths
fdiry = FD_REG
fname = "description.tsv"
fpath = file.path(fdiry, fname)

### read table
dat = read_tsv(fpath, show_col_types = FALSE)
vec = dat$Name
vec[1:3] = paste(vec[1:3], "Peak", sep="_")

### assign and show
vec_txt_cnames_reg = vec
dat

Name,Description
<chr>,<chr>
Chrom,Chromosome
Start,Start
End,End
Assay_Type,Assay type
Assay_Label,Assay label


In [8]:
### set file paths
fdiry = FD_ANT
fname = "description.tsv"
fpath = file.path(fdiry, fname)

### read table
dat = read_tsv(fpath, show_col_types = FALSE)
vec = dat$Name

### assign and show
vec_txt_cnames_ant = vec
dat

Name,Description
<chr>,<chr>
Chrom,Chromosome
Start,Start position
End,End position
Name,Name
Score,Score
Strand,[+-.]; Use '.' if no strand is assigned.
ThickStart,The starting position at which the feature is drawn thickly
ThickEnd,The ending position at which the feature is drawn thickly
ItemRgb,"An RGB value of the form R,G,B (e.g. 255,0,0)"


In [9]:
### init: set column names
cnames = c(
    paste(vec_txt_cnames_reg, sep="_"),
    paste(vec_txt_cnames_ant, sep="_"),
    "Overlap"
)

### init: set file path
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "annotation_enhancer_merge",
    "region_active",
    FOLDER)
fname  = paste("peak.annotation", "bed.gz", sep="*")
fglob  = file.path(fdiry, fname)
fpaths = Sys.glob(fglob)
fnames = basename(fpaths)

### init: annotation and label
annotations = fnames %>% 
    str_split(., "\\.") %>%
    do.call(rbind, .) %>%
    as.data.frame %>%
    dplyr::pull(3)
names(fnames) = annotations
names(annotations) = annotations
print(fnames)

                         ccre_v3                          ccre_v4 
"peak.annotation.ccre_v3.bed.gz" "peak.annotation.ccre_v4.bed.gz" 


In [10]:
### Import data
lst = lapply(annotations, function(annotation){
    ### set file path
    fname = fnames[annotation]
    fpath = file.path(fdiry, fname)
    
    ### read data
    dat = read_tsv(fpath, col_names = cnames, show_col_types = FALSE)
    dat = dat %>% dplyr::mutate(
        Name_Peak  = paste0(Chrom_Peak, ":", Start_Peak, "-", End_Peak),
        Annotation = annotation,
        Label      = Category,
        Region     = paste0(Chrom, ":", Start, "-", End),
        Value      = Score,
        Note       = paste0(Name, ":", Category)
    )
    return(dat)
})

### assign and show
lst_peak_annot_import = lst
print(length(lst))
cat("=========================\n")
print(names(lst))
cat("=========================\n")
head(lst[[1]])

[1] 2
[1] "ccre_v3" "ccre_v4"


Chrom_Peak,Start_Peak,End_Peak,Assay_Type,Assay_Label,Chrom,Start,End,Name,Score,⋯,ThickEnd,ItemRgb,Category,Note,Overlap,Name_Peak,Annotation,Label,Region,Value
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
chr1,180982,182087,ATAC,ATAC,chr1,181251,181601,EH38E1310153,0,⋯,181601,"6,218,147",DNase-only,EH38E1310153:DNase-only,350,chr1:180982-182087,ccre_v3,DNase-only,chr1:181251-181601,0
chr1,777949,779437,"ASTARR,ATAC,ENCODE-E2G_Prediction,LMPRA,WSTARR","ASTARR_A,ATAC,ENCODE-E2G_Prediction:FAM87B,ENCODE-E2G_Prediction:LINC00115,ENCODE-E2G_Prediction:LINC01128,ENCODE-E2G_Prediction:LOC100288069,ENCODE-E2G_Prediction:SAMD11,LMPRA_A,WSTARR_A,WSTARR_AB",chr1,778562,778912,EH38E1310158,0,⋯,778912,"255,0,0","PLS,CTCF-bound","EH38E1310158:PLS,CTCF-bound",350,chr1:777949-779437,ccre_v3,"PLS,CTCF-bound",chr1:778562-778912,0
chr1,777949,779437,"ASTARR,ATAC,ENCODE-E2G_Prediction,LMPRA,WSTARR","ASTARR_A,ATAC,ENCODE-E2G_Prediction:FAM87B,ENCODE-E2G_Prediction:LINC00115,ENCODE-E2G_Prediction:LINC01128,ENCODE-E2G_Prediction:LOC100288069,ENCODE-E2G_Prediction:SAMD11,LMPRA_A,WSTARR_A,WSTARR_AB",chr1,779086,779355,EH38E1310159,0,⋯,779355,"255,0,0",PLS,EH38E1310159:PLS,269,chr1:777949-779437,ccre_v3,PLS,chr1:779086-779355,0
chr1,816774,817620,"ATAC,ENCODE-E2G_Prediction","ATAC,ENCODE-E2G_Prediction:FAM87B,ENCODE-E2G_Prediction:LINC00115,ENCODE-E2G_Prediction:LINC01128",chr1,817080,817403,EH38E1310166,0,⋯,817403,"225,225,225",Low-DNase,EH38E1310166:Low-DNase,323,chr1:816774-817620,ccre_v3,Low-DNase,chr1:817080-817403,0
chr1,817905,818348,ATAC,ATAC,chr1,817903,818252,EH38E1310167,0,⋯,818252,"225,225,225",Low-DNase,EH38E1310167:Low-DNase,347,chr1:817905-818348,ccre_v3,Low-DNase,chr1:817903-818252,0
chr1,818602,819380,"ATAC,WSTARR","ATAC,WSTARR_A",chr1,818718,818872,EH38E1310168,0,⋯,818872,"225,225,225",Low-DNase,EH38E1310168:Low-DNase,154,chr1:818602-819380,ccre_v3,Low-DNase,chr1:818718-818872,0


## Arrange and summarize

In [11]:
### init
lst = lst_peak_annot_import

lst = lapply(lst, function(dat){
    ### summarize
    dat = dat %>%
        dplyr::group_by(Chrom_Peak, Start_Peak, End_Peak, Name_Peak, Assay_Type, Assay_Label, Annotation, Label) %>%
        dplyr::summarise(
            Count  = n(), 
            Region = paste(Region, collapse="|"),
            Score  = mean(Value),
            Note   = paste(Note, collapse="|"),
            .groups = "drop")
    
    ### Arrange
    dat = dat %>% 
        dplyr::arrange(Chrom_Peak,  Start_Peak,  End_Peak) %>%
        dplyr::select(-Chrom_Peak, -Start_Peak, -End_Peak) %>%
        dplyr::rename("Peak" = Name_Peak)
    return(dat)
})

### assign and show
lst_peak_annot_summary = lst
print(length(lst))
cat("=========================\n")
print(names(lst))
cat("=========================\n")
head(lst[[1]])

[1] 2
[1] "ccre_v3" "ccre_v4"


Peak,Assay_Type,Assay_Label,Annotation,Label,Count,Region,Score,Note
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<chr>
chr1:180982-182087,ATAC,ATAC,ccre_v3,DNase-only,1,chr1:181251-181601,0,EH38E1310153:DNase-only
chr1:777949-779437,"ASTARR,ATAC,ENCODE-E2G_Prediction,LMPRA,WSTARR","ASTARR_A,ATAC,ENCODE-E2G_Prediction:FAM87B,ENCODE-E2G_Prediction:LINC00115,ENCODE-E2G_Prediction:LINC01128,ENCODE-E2G_Prediction:LOC100288069,ENCODE-E2G_Prediction:SAMD11,LMPRA_A,WSTARR_A,WSTARR_AB",ccre_v3,PLS,1,chr1:779086-779355,0,EH38E1310159:PLS
chr1:777949-779437,"ASTARR,ATAC,ENCODE-E2G_Prediction,LMPRA,WSTARR","ASTARR_A,ATAC,ENCODE-E2G_Prediction:FAM87B,ENCODE-E2G_Prediction:LINC00115,ENCODE-E2G_Prediction:LINC01128,ENCODE-E2G_Prediction:LOC100288069,ENCODE-E2G_Prediction:SAMD11,LMPRA_A,WSTARR_A,WSTARR_AB",ccre_v3,"PLS,CTCF-bound",1,chr1:778562-778912,0,"EH38E1310158:PLS,CTCF-bound"
chr1:816774-817620,"ATAC,ENCODE-E2G_Prediction","ATAC,ENCODE-E2G_Prediction:FAM87B,ENCODE-E2G_Prediction:LINC00115,ENCODE-E2G_Prediction:LINC01128",ccre_v3,Low-DNase,1,chr1:817080-817403,0,EH38E1310166:Low-DNase
chr1:817905-818348,ATAC,ATAC,ccre_v3,Low-DNase,1,chr1:817903-818252,0,EH38E1310167:Low-DNase
chr1:818602-819380,"ATAC,WSTARR","ATAC,WSTARR_A",ccre_v3,Low-DNase,1,chr1:818718-818872,0,EH38E1310168:Low-DNase


## Explore and check results

In [12]:
lst = lst_peak_annot_summary
lst = lapply(lst, function(dat){
    res = table(dat$Count)
    return(res)
})
print(lst)

$ccre_v3

     1      2      3      4      5      6      7      8      9     10     11 
124267  29317   9023   3041   1126    430    170     62     18      7      3 

$ccre_v4

     1      2      3      4      5      6      7      8      9     10     11 
186509  49233  18049   6966   2876   1275    538    221     94     41     20 
    12     13     14 
     9      3      3 



## Save results

In [13]:
fdiry = file.path(
    FD_REG, 
    "region_active", 
    "summary")

lst = lst_peak_annot_summary
for (idn in names(lst)){
    fname = paste("peak.summary", idn,"tsv", sep=".")
    fpath = file.path(fdiry, fname)
    print(fname)
    flush.console()
    
    dat = lst[[idn]]
    write_tsv(dat, fpath)
}

[1] "peak.summary.ccre_v3.tsv"
[1] "peak.summary.ccre_v4.tsv"


In [24]:
### set file paths
fdiry = file.path(FD_REG, FOLDER)
fname = "region_active.annotation.ccre_v4.bed.gz"
fpath = file.path(fdiry, fname)

### read table
vec_txt_cnames = c(
    paste(vec_txt_cnames_reg, "Region", sep="_"),
    paste(vec_txt_cnames_ant, "Annot",  sep="_"),
    "Overlap"
)
dat = read_tsv(fpath, col_names = vec_txt_cnames)

### arrange table
txt_annotation = "ccre_v4"
dat = dat %>% dplyr::mutate(
    Peak       = paste0(Chrom_Region, ":", Start_Region, "-", End_Region),
    Annotation = txt_annotation,
    Label      = Category_Annot,
    Region     = paste0(Chrom_Annot, ":", Start_Annot, "-", End_Annot),
    Value      = Score_Annot,
    Note       = paste0(Name_Annot, ":", Category_Annot)
)
head(dat)

Rows: 396215 Columns: 17
── Column specification ──────────────────────────────────────────────────────
Delimiter: "\t"
chr (9): Chrom_Region, Assay_Type_Region, Assay_Label_Region, Chrom_Annot, N...
dbl (8): Start_Region, End_Region, Start_Annot, End_Annot, Score_Annot, Thic...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Chrom_Region,Start_Region,End_Region,Assay_Type_Region,Assay_Label_Region,Chrom_Annot,Start_Annot,End_Annot,Name_Annot,Score_Annot,⋯,ItemRgb_Annot,Category_Annot,Note_Annot,Overlap,Peak,Annotation,Label,Region,Value,Note
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
chr1,10015,10442,ATAC,ATAC,chr1,10033,10250,EH38E2776516,0,⋯,"225,225,225",Low-DNase,All-data/Full-classification,217,chr1:10015-10442,ccre_v4,Low-DNase,chr1:10033-10250,0,EH38E2776516:Low-DNase
chr1,16015,16477,ATAC,ATAC,chr1,16097,16381,EH38E3951272,0,⋯,"225,225,225",Low-DNase,All-data/Full-classification,284,chr1:16015-16477,ccre_v4,Low-DNase,chr1:16097-16381,0,EH38E3951272:Low-DNase
chr1,17237,17772,ATAC,ATAC,chr1,17343,17642,EH38E3951273,0,⋯,"225,225,225",Low-DNase,All-data/Full-classification,299,chr1:17237-17772,ccre_v4,Low-DNase,chr1:17343-17642,0,EH38E3951273:Low-DNase
chr1,28903,29613,ATAC,ATAC,chr1,29320,29517,EH38E3951274,0,⋯,"225,225,225",Low-DNase,All-data/Full-classification,197,chr1:28903-29613,ccre_v4,Low-DNase,chr1:29320-29517,0,EH38E3951274:Low-DNase
chr1,115411,115986,"ATAC,ENCODE-E2G_Prediction","ATAC,ENCODE-E2G_Prediction:SAMD11",chr1,115562,115912,EH38E3951280,0,⋯,"0,176,240",CA-CTCF,All-data/Full-classification,350,chr1:115411-115986,ccre_v4,CA-CTCF,chr1:115562-115912,0,EH38E3951280:CA-CTCF
chr1,118518,118743,ATAC,ATAC,chr1,118508,118826,EH38E3951281,0,⋯,"225,225,225",Low-DNase,All-data/Full-classification,225,chr1:118518-118743,ccre_v4,Low-DNase,chr1:118508-118826,0,EH38E3951281:Low-DNase


In [6]:
### init: set column names
cnames = dat_cnames$Name
cnames = c("Chrom_Peak", "Start_Peak", "End_Peak", cnames, "Overlap")

### init: set file path
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "KS91_K562_ASTARRseq_peak_macs_input", 
    FOLDER)
fname  = paste("peak.annotation", "bed.gz", sep="*")
fglob  = file.path(fdiry, fname)
fpaths = Sys.glob(fglob)
fnames = basename(fpaths)

### init: annotation and label
annotations = fnames %>% 
    str_split(., "\\.") %>%
    do.call(rbind, .) %>%
    as.data.frame %>%
    dplyr::pull(3)
names(fnames) = annotations
names(annotations) = annotations
print(fnames)

                         ccre_v3                          ccre_v4 
"peak.annotation.ccre_v3.bed.gz" "peak.annotation.ccre_v4.bed.gz" 


In [8]:
### Import data
lst = lapply(annotations, function(annotation){
    ### set file path
    fname = fnames[annotation]
    fpath = file.path(fdiry, fname)
    
    ### read data
    dat = read_tsv(fpath, col_names = cnames, show_col_types = FALSE)
    dat = dat %>% dplyr::mutate(
        Peak       = paste0(Chrom_Peak, ":", Start_Peak, "-", End_Peak),
        Annotation = annotation,
        Label      = Category,
        Region     = paste0(Chrom, ":", Start, "-", End),
        Value      = Score,
        Note       = paste0(Name, ":", Category)
    )
    return(dat)
})

### assign and show
lst_peak_annot_import = lst
print(length(lst))
cat("=========================\n")
print(names(lst))
cat("=========================\n")
head(lst[[1]])

[1] 2
[1] "ccre_v3" "ccre_v4"


Chrom_Peak,Start_Peak,End_Peak,Chrom,Start,End,Name,Score,Strand,ThickStart,ThickEnd,ItemRgb,Category,Note,Overlap,Peak,Annotation,Label,Region,Value
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
chr1,180982,182087,chr1,181251,181601,EH38E1310153,0,.,181251,181601,"6,218,147",DNase-only,EH38E1310153:DNase-only,350,chr1:180982-182087,ccre_v3,DNase-only,chr1:181251-181601,0
chr1,777949,779437,chr1,778562,778912,EH38E1310158,0,.,778562,778912,"255,0,0","PLS,CTCF-bound","EH38E1310158:PLS,CTCF-bound",350,chr1:777949-779437,ccre_v3,"PLS,CTCF-bound",chr1:778562-778912,0
chr1,777949,779437,chr1,779086,779355,EH38E1310159,0,.,779086,779355,"255,0,0",PLS,EH38E1310159:PLS,269,chr1:777949-779437,ccre_v3,PLS,chr1:779086-779355,0
chr1,816774,817547,chr1,817080,817403,EH38E1310166,0,.,817080,817403,"225,225,225",Low-DNase,EH38E1310166:Low-DNase,323,chr1:816774-817547,ccre_v3,Low-DNase,chr1:817080-817403,0
chr1,817905,818348,chr1,817903,818252,EH38E1310167,0,.,817903,818252,"225,225,225",Low-DNase,EH38E1310167:Low-DNase,347,chr1:817905-818348,ccre_v3,Low-DNase,chr1:817903-818252,0
chr1,818602,819380,chr1,818718,818872,EH38E1310168,0,.,818718,818872,"225,225,225",Low-DNase,EH38E1310168:Low-DNase,154,chr1:818602-819380,ccre_v3,Low-DNase,chr1:818718-818872,0


In [9]:
summary(lst[[1]]$Score)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      0       0       0       0       0       0 

## Arrange and summarize

In [10]:
### init
lst = lst_peak_annot_import

lst = lapply(lst, function(dat){
    ### summarize
    dat = dat %>%
        dplyr::group_by(Chrom_Peak, Start_Peak, End_Peak, Peak, Annotation, Label) %>%
        dplyr::summarise(
            Count  = n(), 
            Region = paste(Region, collapse="|"),
            Score  = mean(Value),
            Note   = paste(Note, collapse="|"),
            .groups = "drop")
    
    ### Arrange
    dat = dat %>% 
        dplyr::arrange(Chrom_Peak,  Start_Peak,  End_Peak) %>%
        dplyr::select(-Chrom_Peak, -Start_Peak, -End_Peak)
    return(dat)
})

### assign and show
lst_peak_annot_summary = lst
print(length(lst))
cat("=========================\n")
print(names(lst))
cat("=========================\n")
head(lst[[1]])

[1] 2
[1] "ccre_v3" "ccre_v4"


Peak,Annotation,Label,Count,Region,Score,Note
<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<chr>
chr1:180982-182087,ccre_v3,DNase-only,1,chr1:181251-181601,0,EH38E1310153:DNase-only
chr1:777949-779437,ccre_v3,PLS,1,chr1:779086-779355,0,EH38E1310159:PLS
chr1:777949-779437,ccre_v3,"PLS,CTCF-bound",1,chr1:778562-778912,0,"EH38E1310158:PLS,CTCF-bound"
chr1:816774-817547,ccre_v3,Low-DNase,1,chr1:817080-817403,0,EH38E1310166:Low-DNase
chr1:817905-818348,ccre_v3,Low-DNase,1,chr1:817903-818252,0,EH38E1310167:Low-DNase
chr1:818602-819380,ccre_v3,Low-DNase,1,chr1:818718-818872,0,EH38E1310168:Low-DNase


## Explore and check results

In [11]:
lst = lst_peak_annot_summary
lst = lapply(lst, function(dat){
    res = table(dat$Count)
    return(res)
})
print(lst)

$ccre_v3

     1      2      3      4      5      6      7      8      9     10     11 
106905  27655   8345   2711    915    356    120     51     14      6      2 

$ccre_v4

     1      2      3      4      5      6      7      8      9     10     11 
156242  47366  17294   6408   2429   1051    450    176     79     32     15 
    12     13     14 
     7      4      1 



## Save results

In [12]:
fdiry = file.path(
    FD_RES, 
    "results", 
    "region", 
    "annotation_enhancer_merge", 
    "summary_starrmpra_crispri_atac")

lst = lst_peak_annot_summary
for (idn in names(lst)){
    fname = paste("peak.summary", idn,"tsv", sep=".")
    fpath = file.path(fdiry, fname)
    print(fname)
    flush.console()
    
    dat = lst[[idn]]
    write_tsv(dat, fpath)
}

[1] "peak.summary.ccre_v3.tsv"
[1] "peak.summary.ccre_v4.tsv"
